In [89]:
# from preprocessing import tokenize, export_to_csv
from gsdmm import MovieGroupProcess
# from topic_allocation import top_words, topic_attribution
# from visualisation import plot_topic_notebook, save_topic_html
from sklearn.datasets import fetch_20newsgroups

import pickle
import matplotlib as plt
import pandas as pd
import numpy as np
from scipy import stats
import ast
import os, re, itertools

import spacy


# Short-text topic modeling on ML paper titles

#### (PILOT DATASET) 20NewsGroups

Note: loadtime is intense!

In [2]:
# cats = ['talk.politics.mideast', 'comp.windows.x', 'sci.space']

# newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
# newsgroups_train_subject = fetch_20newsgroups(subset='train', categories=cats)

# data = newsgroups_train.data
# data_subject = newsgroups_train_subject.data

# targets = newsgroups_train.target.tolist()
# target_names = newsgroups_train.target_names

# # Let's see if our topics are evenly distributed
# df_targets = pd.DataFrame({'targets': targets})
# order_list = df_targets.targets.value_counts()
# order_list

In [5]:
# def extract_first_sentence(data_subject):
#     list_first_sentence = []
#     for text icats = ['talk.politics.mideast', 'comp.windows.x', 'sci.space']

# newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
# newsgroups_train_subject = fetch_20newsgroups(subset='train', categories=cats)

# data = newsgroups_train.data
# data_subject = newsgroups_train_subject.data

# targets = newsgroups_train.target.tolist()
# target_names = newsgroups_train.target_names

# # Let's see if our topics are evenly distributed
# df_targets = pd.DataFrame({'targets': targets})
# order_list = df_targets.targets.value_counts()
# order_listn data:
#         first_sentence = text.split(".")[0].replace("\n", "")
#         list_first_sentence.append(first_sentence)
#     return list_first_sentence

# def extract_subject(data):
#     c = 0
#     s = "Subject:"
#     list_subjects = []
#     for new in data_subject:    
#         lines = new.split("\n")
#         b = 0 # loop out at the first "Subject:", they may be several and we want first one only
#         for line in lines:
#             if s in line and b == 0:
#                 subject = " ".join(line.split(":")[1:]).strip()
#                 subject = subject.replace('Re', '').strip()
#                 list_subjects.append(subject)
#                 c += 1
#                 b = 1
#     return list_subjects
    
# def concatenate(list_first_sentence, list_subjects):
#     list_docs = []
#     for i in range(len(list_first_sentence)):
#         list_docs.append(list_subjects[i] + " " + list_first_sentence[i])
#     return list_docs


# list_first_sentence = extract_first_sentence(data)
# list_subjects = extract_subject(data_subject)
# list_docs = concatenate(list_first_sentence, list_subjects)

# df = pd.DataFrame(columns=['content', 'topic_id', 'topic_true_name'])
# df['content'] = list_docs
# df['topic_id'] = targets

# def true_topic_name(x, target_names):
#     return target_names[x].split('.')[-1]

# df['topic_true_name'] = df['topic_id'].apply(lambda x: true_topic_name(x, target_names))
# df.head()

,content,topic_id,topic_true_name
0,Elevator to the top floor Reading from a Amoco...,1,space
1,"Title for XTerm Yet again,the escape sequences...",0,x
2,From Israeli press. Madness. Before getting ex...,2,mideast
3,Accounts of Anti-Armenian Human Right Violatio...,2,mideast
4,How many israeli soldiers does it take to kill...,2,mideast


#### ML journal dataset

In [68]:
originaldf = pd.read_excel(os.path.expanduser("data.xlsx"))

df = pd.DataFrame({"content": df.title[~df.title.isna()].unique()})
df.head()

,content
0,Computational Perspectives on Social Good and ...
1,A Computational Model of Commonsense Moral Dec...
2,Ethical Challenges in Data-Driven Dialogue Sys...
3,Exploiting Moral Values to Choose the Right Norms
4,"Norms, Rewards, and the Intentional Stance"


## Tokenization & preprocessing

In [15]:
import spacy

nlp = spacy.load("en_core_web_sm")
print(nlp.pipeline)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x000001841016C0A8>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x000001841016C408>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x000001841027A208>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x000001841027A048>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x00000184136D0C88>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x00000184153AA548>)]


In [69]:
# create documents for all titles
docs = list(nlp.pipe(df.content))

# lemmatize each title
def lemmatize(doc):
    # Take the `token.lemma_` of each non-stop word
    L = [re.sub(r"[^a-zA-Z0-9]", r"", token.lemma_.lower()) for token in doc if not (token.is_stop or token.is_punct)]
    L = list(filter(None, L))
    return L
df['tokens'] = list(map(lemmatize, docs))
df.head()

,content,tokens
0,Computational Perspectives on Social Good and ...,"[computational, perspectives, social, good, ac..."
1,A Computational Model of Commonsense Moral Dec...,"[computational, model, commonsense, moral, dec..."
2,Ethical Challenges in Data-Driven Dialogue Sys...,"[ethical, challenge, data, driven, dialogue, s..."
3,Exploiting Moral Values to Choose the Right Norms,"[exploit, moral, value, choose, right, norm]"
4,"Norms, Rewards, and the Intentional Stance","[norm, rewards, intentional, stance]"


In [70]:
print("Max number of token:", df.tokens.apply(len).max())
print("Mean number of token:", df.tokens.apply(len).mean())

# Input format for the model : list of strings (list of tokens)
docs = df.tokens.to_list()
vocab = set(itertools.chain.from_iterable(docs))
n_terms = len(vocab)

print("Voc size:", n_terms)
print("Number of documents:", len(docs))

Max number of token: 17
Mean number of token: 6.2511230907457325
Voc size: 4432
Number of documents: 4452


## Training 

> The value of β thus affects the granularity of the model: a corpus of documents can be sensibly factorized into a set of topics at several different scales, and the particular scale assessed by the model will be set by β. With scientific documents, a large value of β would lead the model to find a relatively small number of topics, perhaps at the level of scientific disciplines, whereas smaller values of β will produce more topics that address specific areas of research.

Finding scientific topics, Thomas L. Griffiths and Mark Steyvers, PNAS 2004

In [96]:
# Train a new model 

# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm
mgp = MovieGroupProcess(K=20, alpha=0.2, beta=0.05, n_iters=50)

vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
n_docs = len(docs)

# Fit the model on the data given the chosen seeds
y = mgp.fit(docs, n_terms)

# Save model
with open('data/gsdmm.model', "wb") as f:
    pickle.dump(mgp, f)
    f.close()

In stage 0: transferred 3738 clusters with 20 clusters populated
In stage 1: transferred 2296 clusters with 20 clusters populated
In stage 2: transferred 1688 clusters with 20 clusters populated
In stage 3: transferred 1400 clusters with 20 clusters populated
In stage 4: transferred 1272 clusters with 20 clusters populated
In stage 5: transferred 1238 clusters with 20 clusters populated
In stage 6: transferred 1136 clusters with 20 clusters populated
In stage 7: transferred 1068 clusters with 20 clusters populated
In stage 8: transferred 1076 clusters with 20 clusters populated
In stage 9: transferred 1046 clusters with 20 clusters populated
In stage 10: transferred 972 clusters with 20 clusters populated
In stage 11: transferred 947 clusters with 20 clusters populated
In stage 12: transferred 941 clusters with 20 clusters populated
In stage 13: transferred 901 clusters with 20 clusters populated
In stage 14: transferred 943 clusters with 20 clusters populated
In stage 15: transferred 

In [97]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topics :', doc_count)
print('*'*20)

# Topics sorted by document inside
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)

# Show the top 5 words by cluster, it helps to make the topic_dict below
for i in top_index:
    ordered = {k: v for k, v in sorted(mgp.cluster_word_distribution[i].items(), key=lambda item: item[1], reverse=True)}
    print("Topic %d: " % i, list(ordered.items())[:5], "\n", "-"*29)

Number of documents per topics : [169  96  97 200  88 119 284 133 317 300 268  83 333 106 503 151 326 201
 256 422]
********************
Most important clusters (by number of docs inside): [14 19 12 16  8  9  6 10 18 17]
********************
Topic 14:  [('learning', 72), ('regression', 53), ('estimation', 46), ('optimal', 41), ('robust', 37)] 
 -----------------------------
Topic 19:  [('learning', 87), ('neural', 78), ('gaussian', 60), ('inference', 50), ('processes', 47)] 
 -----------------------------
Topic 12:  [('learning', 143), ('deep', 62), ('networks', 39), ('neural', 36), ('learn', 35)] 
 -----------------------------
Topic 16:  [('learning', 88), ('regret', 42), ('optimization', 38), ('online', 36), ('optimal', 31)] 
 -----------------------------
Topic 8:  [('learning', 165), ('reinforcement', 74), ('policy', 37), ('multi', 31), ('model', 31)] 
 -----------------------------
Topic 9:  [('neural', 145), ('networks', 103), ('network', 77), ('deep', 67), ('training', 35)] 
 -

$$H[\mathbf{p}]=-\sum_i p_i\log p_i=\sum_i p_i\log 1/p_i$$

In [99]:
if "topicprobs" in df.columns:
    df = df.drop(labels=["topicprobs", "entropy"], axis="columns")
df.insert(2, "topicprobs", df.tokens.apply(mgp.score))
df.insert(3, "entropy", df.topicprobs.apply(stats.entropy))
df.head()

,content,tokens,topicprobs,entropy
0,Computational Perspectives on Social Good and ...,"[computational, perspectives, social, good, ac...","[1.7840914238946043e-10, 1.8815398684715577e-0...",0.000004
1,A Computational Model of Commonsense Moral Dec...,"[computational, model, commonsense, moral, dec...","[7.753955514186043e-08, 1.37399502369604e-12, ...",0.000017
2,Ethical Challenges in Data-Driven Dialogue Sys...,"[ethical, challenge, data, driven, dialogue, s...","[1.463508575219674e-11, 1.5588810036292163e-08...",0.000175
3,Exploiting Moral Values to Choose the Right Norms,"[exploit, moral, value, choose, right, norm]","[3.4714184616619764e-10, 1.4121115736217413e-0...",0.000341
4,"Norms, Rewards, and the Intentional Stance","[norm, rewards, intentional, stance]","[0.0008505911376271891, 3.943614451884207e-06,...",0.036749


In [106]:
df.to_pickle("data/title_topicmodels.gz")